In [1]:
from jetcam.csi_camera import CSICamera
# from jetracer.nvidia_racecar import NvidiaRacecar

import numpy as np
import cv2
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=300, height=300)
print("camera_connected")

# car = NvidiaRacecar()
# print("car connected")

image_widget = ipywidgets.Image(format='jpeg')
print("image widget initialized")

camera_connected
image widget initialized


In [ ]:
import numpy as np
import cv2
camera.running = False
display(image_widget)

objp = np.zeros((7 * 7, 3), np.float32)
objp[:, :2] = np.mgrid[0:7, 0:7].T.reshape(-1, 2)
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# Arrays to store object points and image points from all the images.
imgpoints = []  # 2d points in image plane.
objpoints = []
done = False
while not done:
    img = camera.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret = False
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7, 7))
    # If found, add object points, image points (after refining them)
    if ret == True:
        cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners)
        objpoints.append(objp)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7, 7), corners, ret)
        print(len(imgpoints))
    if len(imgpoints) > 50:
        done = True
    update_image_widget(img)
print("done collecting")
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
from tempfile import TemporaryFile
outfile = TemporaryFile()
np.savez("camera_matrix_dist.npz",mtx,dist)
print("done calibrating")
while True:
    img = camera.read()
    h,  w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0, (w,h))

    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
    # x, y, w, h = roi
    # dst = dst[y:y+h, x:x+w]
    update_image_widget(dst)


In [6]:
h,  w = (300,300)
loaded_arrays = np.load('camera_matrix_dist.npz')
mtx = loaded_arrays["arr_0"]
dist = loaded_arrays["arr_1"]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0, (w,h))



def update_image_widget(change):
    frame = change['new']
    frame = undistort_frame(frame)
    frame = perspective_shift(frame)
    # frame = change
    tl, bl, tr, br = get_corners()
    cv2.circle(frame, tl, 5, (0, 0, 255), -1)
    cv2.circle(frame, bl, 5, (0, 0, 255), -1)
    cv2.circle(frame, tr, 5, (0, 0, 255), -1)
    cv2.circle(frame, br, 5, (0, 0, 255), -1)
    image_widget.value = bgr8_to_jpeg(frame)

In [8]:
def undistort_frame(frame):
    # frame = cv2.undistort(frame, mtx, dist, None, newcameramtx)
    return frame

In [4]:

def get_corners():
    tl = (85, 50)
    bl = (0, 300)
    tr = (215, 50)
    br = (300, 300)
    return tl, bl, tr, br

In [5]:
tl, bl, tr, br = get_corners()
pts1 = np.float32([tl, bl, tr, br])
pts2 = np.float32([[0, 0], [0, 300], [300, 0], [300, 300]])
matrix = cv2.getPerspectiveTransform(pts1, pts2)
def perspective_shift(frame):
    frame = cv2.warpPerspective(frame, matrix, (300, 300))
    return frame

In [7]:
display(image_widget)
camera.running = True
camera.observe(update_image_widget, names='value')

Image(value=b'', format='jpeg')

In [ ]:
camera.unobserve(update_image_widget, names='value')

In [ ]:
loaded_arrays = np.load('camera_matrix_dist.npz')
mtx = loaded_arrays["arr_0"]
dist = loaded_arrays["arr_1"]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0, (w,h))
